In [1]:
import json

In [2]:
file = open('arqJson.json')
dfJsonSempreGas = json.load(file)

In [3]:
dfJsonSempreGas

{'cylinders': {'tabelaSQLServer': 'public.cylinders',
  'colunasTabela': 'id,device_id,tare,created_at,updated_at,first_reading_id,historical_consumption,address_id,seal_number,installed,manual_last_reading_weight,brand,usage',
  'tabela_destino': 'tmp_cylinders',
  'tabela_origem': 'cylinders',
  'colunasMerge': 'id,id2'},
 'consumption_rates': {'tabelaSQLServer': 'public.consumption_rates',
  'colunasTabela': 'id, client_id, device_id, days_until_empty, rate, created_at, updated_at',
  'tabela_destino': 'tmp_consumption_rates',
  'tabela_origem': 'consumption_rates',
  'colunasMerge': 'id'}}

In [5]:
for i in dfJsonSempreGas:
    tabelaSQLServer = dfJsonSempreGas[i]['tabelaSQLServer']
    colunasTabela = dfJsonSempreGas[i]['colunasTabela']
    tabela_destino = dfJsonSempreGas[i]['tabela_destino']
    tabela_origem = dfJsonSempreGas[i]['tabela_origem']
    colunasMerge = dfJsonSempreGas[i]['colunasMerge']
    
    print('----------------------------------------------------------------------')
    
    print('tabela que será atualizada: ' + '\n' + tabelaSQLServer + '\n') 

    listMerge = [field.strip().split(' ')[0] for field in colunasMerge.split(',')]
    listColunas = [fieldColunas.strip().split(' ')[0] for fieldColunas in colunasTabela.split(',')]

    colunasMerge_lista = []
    colunasUpdate_lista = []
    colunas_insert = []
    
    for cm in listMerge:
        colunasMerge_lista.append('{tabela_destino}.{cm} = {tabela_origem}.{cm}'.format(**locals()))

    merge_final = ' AND '.join(colunasMerge_lista)
    
    print('merge_final: '+ '\n' + merge_final + '\n')

    for colunas in listColunas:
        colunas_insert.append('{tabela_origem}.{colunas}'.format(**locals()))
        if colunas not in listMerge:
            colunasUpdate_lista.append('{tabela_destino}.{colunas} = {tabela_origem}.{colunas}'.format(**locals()))

    colunas_insert_final = ','.join(colunas_insert)
    colunas_final = ','.join(colunasUpdate_lista)

    print('colunas insert: '+ '\n' + colunas_insert_final + '\n')
    
    query = """   
    MERGE INTO {0}
    USING {1}
    ON {2}
    WHEN MATCHED THEN
    UPDATE SET 
    {3}
    WHEN NOT MATCHED
    THEN INSERT ({4}) VALUES ({5})    
    """.format(tabela_destino,tabela_origem,merge_final,colunas_final,colunasTabela,colunas_insert_final)
    
    print('comando merge: ' + '\n' + query + '\n')


----------------------------------------------------------------------
tabela que será atualizada: 
public.cylinders

merge_final: 
tmp_cylinders.id = cylinders.id AND tmp_cylinders.id2 = cylinders.id2

colunas insert: 
cylinders.id,cylinders.device_id,cylinders.tare,cylinders.created_at,cylinders.updated_at,cylinders.first_reading_id,cylinders.historical_consumption,cylinders.address_id,cylinders.seal_number,cylinders.installed,cylinders.manual_last_reading_weight,cylinders.brand,cylinders.usage

comando merge: 
   
    MERGE INTO tmp_cylinders
    USING cylinders
    ON tmp_cylinders.id = cylinders.id AND tmp_cylinders.id2 = cylinders.id2
    WHEN MATCHED THEN
    UPDATE SET 
    tmp_cylinders.device_id = cylinders.device_id,tmp_cylinders.tare = cylinders.tare,tmp_cylinders.created_at = cylinders.created_at,tmp_cylinders.updated_at = cylinders.updated_at,tmp_cylinders.first_reading_id = cylinders.first_reading_id,tmp_cylinders.historical_consumption = cylinders.historical_consumption